In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...


In [2]:
!mamba install -y -c conda-forge rdkit openmm openff-toolkit mdanalysis nglview


Looking for: ['rdkit', 'openmm', 'openff-toolkit', 'mdanalysis', 'nglview']

[+] 0.0s
conda-forge/linux-64  ⣾  [+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64   6%
conda-forge/noarch     1%[+] 0.3s
conda-forge/linux-64  10%
conda-forge/noarch    15%[+] 0.4s
conda-forge/linux-64  13%
conda-forge/noarch    23%[+] 0.5s
conda-forge/linux-64  17%
conda-forge/noarch    40%[+] 0.6s
conda-forge/linux-64  20%
conda-forge/noarch    40%[+] 0.7s
conda-forge/linux-64  20%
conda-forge/noarch    49%[+] 0.8s
conda-forge/linux-64  24%
conda-forge/noarch    49%[+] 0.9s
conda-forge/linux-64  24%
conda-forge/noarch    58%[+] 1.0s
conda-forge/linux-64  28%
conda-forge/noarch    67%[+] 1.1s
conda-forge/linux-64  32%
conda-forge/noarch    67%[+] 1.2s
conda-forge/linux-64  36%
conda-forge/noarch    76%[+] 1.3s
conda-forge/linux-64  40%
conda-forge/noarch    86%[+] 1.4s
conda-forge/linux-64  44%
conda-forge/noarch    95%conda-forge/noarch                              

In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem

epoxy = Chem.MolFromSmiles("CC(C)(C1=CC=C(C=C1)OCC2CO2)C3=CC=C(C=C3)OCC4CO4")
epoxy = AllChem.AddHs(epoxy)
AllChem.EmbedMolecule(epoxy)
Chem.MolToPDBFile(epoxy, "epoxy.pdb")
n_epoxy = 50

diamine = Chem.MolFromSmiles("NCCN")
diamine = AllChem.AddHs(diamine)
AllChem.EmbedMolecule(diamine)
Chem.MolToPDBFile(diamine, "diamine.pdb")
n_diamine = 50

l = 50

packmol_scripts = f"""
tolerance 0.2
filetype pdb
output mixture.pdb
structure epoxy.pdb
  number {n_epoxy}
  inside box 1 1 1 {l-1} {l-1} {l-1}
end structure
structure diamine.pdb
  number {n_diamine}
  inside box 1 1 1 {l-1} {l-1} {l-1}
end structure
"""

with open("packmol.inp", "w") as f:
  f.write(packmol_scripts)

!packmol < packmol.inp


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                           Included as part of Packmol Memgen
                                                              Version 20.14.3 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Types of coordinate files specified: pdb
  Seed for random number generator:      1234567
  Output file: mixture.pdb
  Reading coordinate file: epoxy.pdb
  Reading coordinate file: diamine.pdb
  Number of independent structures:            2
  The structures are: 
  Structure            1 :epoxy.pdb(          49  atoms)
  Structure            2 :diamine.pdb(          12  atoms)
  Maximum number of GENCAN loops f

In [49]:
from openff.interchange import Interchange
from openff.toolkit import ForceField, Molecule, Topology
import numpy as np
from openff.units import unit as off_unit
from openmm import unit as omm_unit

mol = Chem.MolFromPDBFile("mixture.pdb", removeHs=False, sanitize=False, proximityBonding=False)
conf = mol.GetConformer(0)
for conf in mol.GetConformers():
  positions = conf.GetPositions()
positions = Quantity(positions, omm_unit.angstroms)

mols = Chem.GetMolFrags(mol, asMols=True)
off_mols = []
for m in mols:
  off_mol = Molecule.from_rdkit(m, allow_undefined_stereo=True, hydrogens_are_explicit=True)
  off_mol.assign_partial_charges('gasteiger')
  off_mols.append(off_mol)

box = off_unit.Quantity(l * np.eye(3), off_unit.angstrom)
ff = ForceField("openff-2.0.0.offxml")
top = Topology.from_molecules(off_mols)

interchange = Interchange.from_smirnoff(
    topology=top,
    force_field=ff,
    charge_from_molecules=off_mols,
    box=box,
    positions=positions
)

In [50]:
interchange

Interchange with 7 collections, periodic topology with 3050 atoms.

In [53]:
from openmm.app import PDBFile, StateDataReporter, PDBReporter, XTCReporter
from sys import stdout
from openmm.app import Simulation
from openmm import LangevinMiddleIntegrator
from openmm import MonteCarloBarostat

from openmm.unit import Quantity
from openmm.app import HBonds

integrator = LangevinMiddleIntegrator(300*omm_unit.kelvin, 1/omm_unit.picosecond, 0.002*omm_unit.picoseconds)
simulation = interchange.to_openmm_simulation(integrator)
barostat = MonteCarloBarostat(1.0*omm_unit.bar, 300*omm_unit.kelvin, 25)
simulation.system.addForce(barostat)

state = simulation.context.getState(getPositions=True, getEnergy=True, enforcePeriodicBox=True)
PDBFile.writeFile(simulation.topology, state.getPositions(), open('example.pdb', 'w'))
simulation.reporters.append(PDBReporter('example-traj.pdb', 100))
simulation.reporters.append(XTCReporter('example-traj.xtc', 100))
simulation.reporters.append(StateDataReporter(
    stdout,
    100,
    step=True,
    potentialEnergy=True,
    kineticEnergy=True,
    temperature=True,
    volume=True,
    density=True
    )
)

simulation.context.reinitialize(preserveState=True)
simulation.minimizeEnergy()
simulation.step(100_000)

#"Step","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Density (g/mL)"
100,80004.46467666732,1760.6841323931824,56.095795255440244,125.21790019993206,0.2655643295434889
200,81349.74128431405,3124.505214937256,99.54744385265766,125.4892550245858,0.26499007988312206
300,82365.46180386955,4380.621417476808,139.56759057762665,125.1688512135296,0.2656683942613678
400,83343.78764200729,5296.857571678443,168.75908197924804,125.47587488079436,0.2650183371507084
500,84079.70927488856,6137.479061644473,195.54147305149704,126.06538222520606,0.2637790575531181
600,84695.12070429932,6715.406471292852,213.9543712242527,125.48894840467253,0.2649907273603409
700,85109.49323745104,7294.971685575159,232.4194502236104,124.96694509231016,0.2660976283678452
800,85618.67695262106,7618.16393067251,242.71642835590293,123.97068459715227,0.26823605775427267
900,85717.60176877014,7911.256846105293,252.0544350275425,123.94191758125088,0.2682983155528394
1000,85757.9